In [ ]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np
import os

#Function to load, preprocess, and return preprocessed image
def load_preprocess_img(path):
  #load and convert to numpy
  image = load_img(path, target_size=(224, 224))
  image = img_to_array(image)
  #preprocessing - specific to mobilenetv2
  image = preprocess_input(image)
  return image

#LOAD DATASET
IMAGES = '/content/drive/MyDrive/FaceMaskDetector/images'    #path to images
CATEGORIES = ['with_mask', 'without_mask', 'mask_weared_incorrect'] #categories of dataset
DATA_SIZE = [1250, 1250, 2994] #number of images to load for each category

data = []
target = []

#PREPROCESS
for i in range(len(CATEGORIES)):
  category = CATEGORIES[i]
  print(f'Loading {category} images')
  category_path = os.path.join(IMAGES, category)
  for image in os.listdir(category_path)[:DATA_SIZE[i]]:
    image = load_preprocess_img(os.path.join(category_path, image))
    data.append(image)
    target.append(category)


data = np.array(data)
target = np.array(target)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

#ENCODE TARGET VALUES
le = LabelEncoder()
target = le.fit_transform(target)

le.classes_

In [4]:
#TRAIN-VALIDATION-TEST SPLIT
from sklearn.model_selection import train_test_split
#get train data
X_train, X_temp, Y_train, Y_temp = train_test_split(data, target, test_size=0.4, random_state = 4001, stratify=target)
#get validation and test data
X_validate, X_test, Y_validate, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state = 4001)

In [ ]:
#CONFIGURE MOBILENETV2
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input

mobile = MobileNetV2(
    weights='imagenet', #use pretrained weights from imagenet dataset
    include_top = False,
    input_tensor=Input(shape=(224, 224, 3))
)
#Original mobilenet model
mobile.summary()

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam

outputLayer = mobile.output

#add pooling layer to downsample output of mobilenet
outputLayer = AveragePooling2D(pool_size=(7, 7))(outputLayer)
#convert output of pooling layer into linear vector
outputLayer = Flatten(name="flatten")(outputLayer)
#add hidden layer with 248 neurons
outputLayer = Dense(248, activation="relu")(outputLayer)
#add dropout layer to nullify output of previous layer to prevent overfitting
outputLayer = Dropout(0.5)(outputLayer)
#final layer for output consisting of 3 neurons for 3 classes -> masked, unmasked, improper mask
outputLayer = Dense(3, activation="softmax")(outputLayer)

model = Model(inputs=mobile.input, outputs=outputLayer)

#Preserve imagenet weights from original mobilenet, only newly added layer weights will be updated
for layer in mobile.layers:
  layer.trainable = False

#COMPILE MODEL
model.compile(
    optimizer=Adam(learning_rate=0.0001), 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
    )
#Modified model
model.summary()

In [ ]:
#TRAIN MODEL
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#use image data generator to break training data into batches
idg = ImageDataGenerator()

EPOCHS = 20
fitting_data = model.fit(
    idg.flow(X_train, Y_train, batch_size=32),
    validation_data=(X_validate, Y_validate),
    epochs = EPOCHS,
    verbose = 2
)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.plot(fitting_data.history["accuracy"], label="Training accuracy")
plt.plot(fitting_data.history["val_accuracy"], label="Value accuracy")
plt.title("Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(loc="lower left")
plt.subplot(122)
plt.plot(fitting_data.history["loss"], label="Training loss")
plt.plot(fitting_data.history["val_loss"], label="Value loss")
plt.title("Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(loc="lower left")
plt.savefig("plot-model1.png")

In [9]:
#Export model for future use
model.save('mask_detector.model', save_format='h5')

In [ ]:
#ANALYZE ACCURACY
from sklearn.metrics import classification_report

#Predict target values of test data
pred = model.predict(X_test, batch_size=32)

#Set prediction to class with highest probability
pred = np.argmax(pred, axis=1)
print(classification_report(Y_test.tolist(), pred.tolist(), target_names=le.classes_))